In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import gensim
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
# Import SQUINKY sentences
squinky = pd.read_csv("..\..\data\mturk_experiment_2.csv",encoding="unicode escape")
sentences = pd.DataFrame()
sentences["Sentence"] = squinky["Sentence"]

In [3]:
# Clustering
vectoriser = TfidfVectorizer()
vector = vectoriser.fit_transform(sentences["Sentence"])
k = 5
model = KMeans(n_clusters=k,init='k-means++',max_iter=200,n_init=10)
model.fit(vector)
labels = model.labels_.tolist()

# Topic modelling

vectoriser = CountVectorizer(min_df=20, max_df=0.2, stop_words='english',token_pattern="(?u)\\b\\w\\w\\w+\\b")
X = vectoriser.fit_transform(sentences["Sentence"])
corpus = gensim.matutils.Sparse2Corpus(X,documents_columns=False)
id_map = dict((v,k) for k,v in vectoriser.vocabulary_.items())
ldamodel = gensim.models.LdaMulticore(corpus=corpus, id2word=id_map,passes=2,random_state=5,num_topics=10,workers=2)

topic = []

for doc in sentences["Sentence"]:
    X = vectoriser.transform([doc])
    corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)
    output = list(ldamodel[corpus])[0]
    topics = sorted(output, key=lambda x: x[1], reverse=True)
    topic.append(topics[0][0])

In [5]:
sentences["Cluster"] = labels
sentences["Topic"] = topic

#sentences.to_csv("..\..\data\squinky_nlp.csv",index=False)
sentences

,Sentence,Cluster,Topic
0,10In High Bay 4 of the Vehicle Assembly Buildi...,3,1
1,12The oxygen vent arm and hood removed from th...,3,1
2,"13In the Rotation, Processing and Surge Facili...",3,1
3,14In the Vehicle Assembly Building's High Bay ...,3,1
4,15In the Vehicle Assembly Building's High Bay ...,0,1
...,...,...,...
7027,Let's hope it's true.,1,3
7028,"Cheer up, Sony (NYSE: SNE) fans.",0,3
7029,Wired is one of the few publications that acts...,3,9
7030,"As NewsBusters reported Friday, conservative t...",0,5
